# 思考？如何提升成绩

## TFIDF  词的文本  字    那么 词 + 字的特征   逻辑回归
## 逻辑回归  LightGBM Xgboost SVM 

# 读取数据集

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_set.csv')
test = pd.read_csv('./data/test_set.csv')

In [4]:
train.head()

,id,article,word_seg,class
0,0,7368 1252069 365865 755561 1044285 129532 1053...,816903 597526 520477 1179558 1033823 758724 63...,14
1,1,581131 165432 7368 957317 1197553 570900 33659...,90540 816903 441039 816903 569138 816903 10343...,3
2,2,7368 87936 40494 490286 856005 641588 145611 1...,816903 1012629 957974 1033823 328210 947200 65...,12
3,3,299237 760651 299237 887082 159592 556634 7489...,563568 1239563 680125 780219 782805 1033823 19...,13
4,4,7368 7368 7368 865510 7368 396966 995243 37685...,816903 816903 816903 139132 816903 312320 1103...,12


# TFIDF构建文本特征

## 词的TFIDF

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
word_vec = TfidfVectorizer(analyzer='word',
            ngram_range=(1,2),
            min_df=3, 
            max_df=0.9,
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [7]:
train_term_doc = word_vec.fit_transform(train['word_seg'])

In [8]:
test_term_doc = word_vec.transform(test['word_seg'])

## 字的TFIDF

In [9]:
char_vec = TfidfVectorizer(analyzer='word', #char"我们来到深度之眼"
            ngram_range=(1,2), #(1,3)
            min_df=3, 
            max_df=0.9,
            use_idf=True,
            smooth_idf=True, 
            sublinear_tf=True)

In [10]:
train_term_char = char_vec.fit_transform(train['article'])

In [11]:
test_term_char = char_vec.transform(test['article'])

In [12]:
#词
train_term_doc

<102277x2820641 sparse matrix of type '<class 'numpy.float64'>'
	with 75739932 stored elements in Compressed Sparse Row format>

In [13]:
#字
train_term_char #[[0],[1]]

<102277x1241880 sparse matrix of type '<class 'numpy.float64'>'
	with 112400674 stored elements in Compressed Sparse Row format>

In [14]:
from scipy.sparse import hstack
train_term_doc = hstack([train_term_doc, train_term_char])

In [15]:
#102277x4062521 
train_term_doc

<102277x4062521 sparse matrix of type '<class 'numpy.float64'>'
	with 188140606 stored elements in COOrdinate format>

In [16]:
test_term_doc = hstack([test_term_doc, test_term_char])
#

In [17]:
#102277x4062521 
test_term_doc

<102277x4062521 sparse matrix of type '<class 'numpy.float64'>'
	with 185703775 stored elements in COOrdinate format>

In [18]:
train_term_doc,test_term_doc

(<102277x4062521 sparse matrix of type '<class 'numpy.float64'>'
 	with 188140606 stored elements in COOrdinate format>,
 <102277x4062521 sparse matrix of type '<class 'numpy.float64'>'
 	with 185703775 stored elements in COOrdinate format>)

# 逻辑回归构建模型

正确的操作应该是这样的：

In [19]:
from sklearn.preprocessing import LabelEncoder

In [20]:
lb = LabelEncoder()
train['label'] = lb.fit_transform(train['class'].tolist())

In [21]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(C=4, dual=True)
clf.fit(train_term_doc,train['label'])


/home/hcq/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/hcq/miniconda3/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=4, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [22]:
test_prob = clf.predict_proba(test_term_doc)

# 结果提交

In [23]:
test_pred = np.argmax(test_prob,axis=1)
test['class'] = lb.inverse_transform(test_pred)

In [24]:
test[["id","class"]].to_csv("submission_baseline_word_char.csv",index=False,header=True,encoding='utf-8')